#Active Learning Selection of training file

Active learning originaly aims to maximize the information gained from labeling at a limited annotation budget. In this applictaion, as we noticed the classes are highley imbalanced, and the presence of images and large objects all problems thata could hender the model's ability to generalize at deployment.

In this Notebook we present the Weighing by Random Performance Sum of Averages Active learning approach proposed. 



1.   It first evaluate the model from the previous iteration on a random subset of the training data that hasn't been used for trianing yet.
2.   It then calculated the uncertinity of the inference made on the training data that hasn't been used for trianing yet, by summing the average uncertinities per class for eaach image
3.  After that it multiplies the weights with uncertinities and produce the final uncertinity score per image.
4. Finally, the images with the highest uncertinities are included in the new training cycle.
5. I model is build using the selected images using the notebook in [here](https://github.com/asmayamani/GreenHawk/blob/main/GreenHawk_training.ipynb)


* In some iterations we included some of the images in which were not detected at all by the model seetting their uncertinities to +inf
* We also included all images containing the 5 undererepresented classes explicitly.




#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


In [ ]:
import os, glob, random, shutil
from pathlib import Path    

In [ ]:
!git clone https://github.com/asmayamani/ultralytics-AL.git
%cd ultralytics-AL
!pip install -e '.[dev]'


Cloning into 'ultralytics-AL'...
remote: Enumerating objects: 2766, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 2766 (delta 74), reused 51 (delta 14), pack-reused 2582
Receiving objects: 100% (2766/2766), 1.52 MiB | 4.60 MiB/s, done.
Resolving deltas: 100% (1827/1827), done.
/content/ultralytics-AL
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/ultralytics-AL
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Declerations

Important: Increase the itereation after completing each iteration

In [ ]:
#Enter the number iteration
iteration = 16

In [ ]:

main_dir = '/content/drive/MyDrive/Smartathon/'
complete_data_folder = main_dir+'TrainF/'
main_exp_dir = 'WRP/'
iteration_folder = 'Iteration_'+str(iteration)+'/'
complete_iteration_folder = main_dir+main_exp_dir+iteration_folder
os.makedirs(complete_iteration_folder) if not os.path.exists(complete_iteration_folder) else None
object_def = {'graffiti':0,'faded_signage':1,'potholes':2,'garbage':3,
                   'construction_road':4,'broken_signage':5,
                   'bad_streetlight':6,'bad_billboard':7,'sand_on_road':8,'clutter_sidewalk':9,'unkept_facade':10}
approach = 'SumAvg' 
weighting_approach = 'WRP'
number_of_random_images_from_training = 6000
output_model = approach+str(iteration-1)+str(number_of_random_images_from_training)
previous_model_zip = 'Iteration_'+str(iteration-1)+'_W.zip'
budgetPerIteration = 300
expName = 'iteration'+approach+str(iteration)+'_'+str(number_of_random_images_from_training)
wexpName = 'w_iteration'+approach+str(iteration)+'_'+str(number_of_random_images_from_training)
previousExpfName = 'iteration'+str(iteration-1)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)+'/'
previous_iteration_folder = 'Iteration_'+str(iteration-1)+'/'


#Definitions

The methods for active learning and some utility functions are defined here 

In [ ]:
def sumAvgUncertinitiesForFile(fDir, classesWeight):
    classes = {}
    for i in range(len(object_def)):
      classes[str(i)] = {'objectsC' : 0, 'uncer' : 0}
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
              classes[str(row[0])]['objectsC'] += 1
              classes[str(row[0])]['uncer'] += ((1-float(row[5])))
      file2.close()
      total = 0
      wAvg = 0
      for i in range(len(object_def)):
        # print(classes[str(i)])
        if classes[str(i)]['objectsC'] != 0:
          wAvg += ((classes[str(i)]['uncer']/classes[str(i)]['objectsC']) * classesWeight[str(i)])
          total += classes[str(i)]['objectsC'] 

      return(wAvg)
    except (IOError, OSError) as e:
        print(e)
        return(0)


In [ ]:
def getUncertinitiesForDir(dir, trainDir = [], valAccuracyDirF = '', method = 'avg', weighting = 1 , budget = 100):
  classesWeight = getWeightsForClassses(weighting, trainDir = trainDir, valAccuracyDirF = valAccuracyDirF)
  listofFilesNamesUncertinity = {}
  for f in glob.glob(dir+'/labels/*.txt'):
    namef = Path(f).name[:-4]
    # listofFilesNames.append(namef)
    fileFullDir = dir+'/labels/'+namef+'.txt'
    # if method == 'sum':
    #   listofFilesNamesUncertinity[namef] = sumUncertinitiesForFile(fileFullDir,classesWeight)
    # if method == 'avg':
    #   listofFilesNamesUncertinity[namef] = avgUncertinitiesForFile(fileFullDir,classesWeight)
    if method == 'sumAvg':
      listofFilesNamesUncertinity[namef] = sumAvgUncertinitiesForFile(fileFullDir,classesWeight)
  print(listofFilesNamesUncertinity)
  return sorted(listofFilesNamesUncertinity, key=listofFilesNamesUncertinity.get, reverse=True)[:budget]


In [ ]:
def getWeightsForClassses(weighting, trainDir = [], valAccuracyDirF = '', ):
  classesWeight = {}
  if weighting == 1:
    for i in range(len(object_def)):
      classesWeight[str(i)] = 1
  # elif weighting == 2:
  #   classesWeight = getWeightForTrainingLabelsPerClass(trainDir)
  elif weighting == 3:
    classesWeight = applyMinMaxScalerToW(getWeightForValAccuracy(valAccuracyDirF))
  print(classesWeight)
  return classesWeight


In [ ]:
def newTraining(namefolder,setOfFileNames,oldTraning):
  added = 0
  path_label_2 = complete_iteration_folder+namefolder+'/labels/'
  path_images_2 = complete_iteration_folder+namefolder+'/images/'
  os.makedirs(path_label_2) if not os.path.exists(path_label_2) else None
  os.makedirs(path_images_2) if not os.path.exists(path_images_2) else None
  count = 0
  #why 100?
  while (added < budgetPerIteration*2) and (count < len(setOfFileNames)):
    namef = setOfFileNames[count] #Path(setOfFileNames[count]).name[:-4]
    
    l_src_path = complete_data_folder+'labels/'+ namef+'.txt'
    i_src_path = complete_data_folder+'images/'+ namef+'.jpg'

    src_path = complete_data_folder+'labels/'+ namef+'.txt'
    if os.path.exists(l_src_path) and os.path.exists(i_src_path):
      dst_path = path_label_2 + namef+'.txt'
      shutil.copy(src_path, dst_path)
      src_path = complete_data_folder+'images/'+ namef+'.jpg'
      dst_path = path_images_2 + namef+'.jpg'
      shutil.copy(src_path, dst_path)
      added += 1
    count += 1

  for pth in oldTraning:
    # print(pth)
    namef = Path(pth).name[:-4]
    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/'+ namef+'.txt'
    dst_path = path_label_2 + namef+'.txt'
    shutil.copy(src_path, dst_path)
    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'images/'+ namef+'.jpg'
    dst_path = path_images_2 + namef+'.jpg'
    shutil.copy(src_path, dst_path)


In [ ]:
def getWeightForValAccuracy(fDir, cnum = object_def):
    numberOfClasses = len(object_def)
    accuracyClasses = {}
    for i in range(numberOfClasses):
      accuracyClasses[str(i)] = 0
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          first = True
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
            if first:
              first = False 
              continue
            row = list(filter(lambda a: a != '', row))
            accuracyClasses[str(cnum[row[0]])] = float(row[5])
            
            
      file2.close()
      return(accuracyClasses)
    except (IOError, OSError) as e:
        print(e)


In [ ]:
def applyMinMaxScalerToW (wValCDic):
  minV = min(wValCDic[item] for item in wValCDic)
  maxV = max(wValCDic[item] for item in wValCDic)
  for i in range(len(object_def)):
    wValCDic[str(i)] = 1- ((wValCDic[str(i)] - minV)/(maxV - minV))
  return wValCDic

In [ ]:
def nominateImagesToObtainUncer(wasTrainedOnDir, experimentName, forWeights = False, fromRandSize = 500, seed = 42, trainingDir = complete_data_folder+'images/', weightedOnDir = '' ):
  #remove images that were trained on
  trainedOn = glob.glob(wasTrainedOnDir+'/images/*.jpg')
  fullTrainSet = glob.glob(trainingDir+'*.jpg')
  orpNames = [Path(f).name[:-4] for f in trainedOn]
  if weightedOnDir != '':
    weightedOn = glob.glob(weightedOnDir+'/images/*.jpg')
    orpNames = orpNames + [Path(f).name[:-4] for f in weightedOn]

  mixedRand = []
  for ele in fullTrainSet:
    for ele2 in orpNames:
      if ele2 in ele:
        mixedRand.append(ele)
        continue
  randomPathes2 = [ele for ele in fullTrainSet if ele not in mixedRand]

  reducedRandomPathes2 = []
  for rp2 in randomPathes2:
    if os.path.exists(complete_data_folder+'labels/'+ Path(rp2).name[:-4] + '.txt'):
        reducedRandomPathes2.append(rp2)
  toGetUncertinityPath = reducedRandomPathes2
  if fromRandSize < number_of_random_images_from_training:
    toGetUncertinityPath = random.sample(reducedRandomPathes2, k=fromRandSize)

  newFolderName = experimentName+'_Nominees'
  path_label = complete_iteration_folder+newFolderName+'/labels/'
  path_images = complete_iteration_folder+newFolderName+'/images/'
  os.makedirs(path_label) if not os.path.exists(path_label) else None
  os.makedirs(path_images) if not os.path.exists(path_images) else None

  for pth in toGetUncertinityPath:
    namef = Path(pth).name[:-4]
    if forWeights:
      src_path = complete_data_folder+'labels/'+ namef+'.txt'
      dst_path = path_label + namef+'.txt'
      shutil.copy(src_path, dst_path)
    src_path = complete_data_folder+'/images/'+ namef+'.jpg'
    dst_path = path_images + namef+'.jpg'
    shutil.copy(src_path, dst_path)
  return path_images


#Test on Random subset
In this part we select some random validation subset to evaluate the peformance on and subsequently weigh the uncertinitiies of the different classes

In [ ]:
path_images_nominees_test = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,wexpName,fromRandSize=int(budgetPerIteration*0.1),forWeights=True)


7079
4160
['4624ef109293465950b4a44ba5d13e96', '4a8080f6153cc3b7107070fa06535c3a', 'db28fdf0199e22b1ef795d6d4cad35ba', '6ffd85478dcf11d5885af3773a9d6117', 'ec68f3dd632f677a6b397db98257be1e', '594c9da614f0d5864ec01614d6899a86', 'b518ab2d608ba3e124dc421e35519d1c', 'b48fadb3eac7c2dfc1da80be4bc9fbf2', '4d4f1d3acb287df727c2839cac676442', '322701a21f259587f7e92f8690f91aed', 'a87c205a8a7d1b5318a53e46439e3880', 'aff5609101b491d28a9f87b891d90291', '329a011b04f8aef22e063cbfda44df80', 'f8d9736da8ce04e71fec0547b6626051', '6e6fd1f8c1909d9a16ffc50c03180b71', 'ac6629b73c2989969310cc665ea95b0d', '429f04f02f3d259344bf453ed52536f8', '33e22b6ad0088ad51682359f14a62e75', '83274dc535a594f4c42b9dae620766e1', 'b8d2994297561cb66e9d7973503dcad7', 'f6439adc4faa188d4ba30d40296f7e5c', '6c101ec46de80b9f535fc9ba3e04c62d', '181b71e92880d3be02b4d9bda7feaa4c', 'cc9f9bf7fa0488d73ccea787e6166219', '9a0323e220914088050d3a82489c3d16', '1f3e5d2a1e740f71ed1654b339e46384', '216dd17a91843c6ae22fb2767495eb92', 'b37be3c8f19410b3

In [ ]:
!unzip {main_dir+main_exp_dir+previous_iteration_folder}{previous_model_zip} -d /content/ultralytics-AL/ultralytics/{previous_iteration_folder}

Archive:  /content/drive/MyDrive/Smartathon/WRP/Iteration_15/Iteration_15_W.zip
  inflating: /content/ultralytics-AL/ultralytics/Iteration_15/content/runs/detect/train/weights/best.pt  


In [ ]:
#This somtims has to be updated manually based on the unzipped file above
weights_path = previous_iteration_folder+'/content/runs/detect/train/weights/best.pt'

In [ ]:
import yaml
config = {
        #  'path': '../datasets/Smartathon',
        'train': '/content/drive/MyDrive/Smartathon/WRP/Iteration_0/images',
         'val': complete_iteration_folder+wexpName+'_Nominees/images',
         'nc': 11,
         'names': ['graffiti','faded_signage','potholes','garbage',
                   'construction_road','broken_signage',
                   'bad_streetlight','bad_billboard','sand_on_road','clutter_sidewalk','unkept_facade']}

with open("/content/ultralytics-AL/ultralytics/data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)


In [ ]:
%cd ultralytics
!yolo task=detect mode=val data=data.yaml model={weights_path}  

[Errno 2] No such file or directory: 'ultralytics'
/content/ultralytics-AL/ultralytics
Ultralytics YOLOv8.0.9 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /content/drive/MyDrive/Smartathon/WRP/Iteration_16/w_iterationSumAvg16_6000_Nominees/labels.cache... 89 images, 0 backgrounds, 0 corrupt: 100% 89/89 [00:00<?, ?it/s]
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 6/6 [00:36<00:00,  6.12s/it]
                   all         89        215      0.576      0.553      0.549      0.253
              graffiti         89         17      0.797      0.647      0.696       0.41
              potholes         89         14      0.292      0.214      0.208     0.0514
               garbage         89        111      0.5

#Calculating the uncertinity

In [ ]:
path_images_nominees = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,expName, fromRandSize = number_of_random_images_from_training, weightedOnDir=complete_iteration_folder+wexpName+'_Nominees')


7079
4160
['4624ef109293465950b4a44ba5d13e96', '4a8080f6153cc3b7107070fa06535c3a', 'db28fdf0199e22b1ef795d6d4cad35ba', '6ffd85478dcf11d5885af3773a9d6117', 'ec68f3dd632f677a6b397db98257be1e', '594c9da614f0d5864ec01614d6899a86', 'b518ab2d608ba3e124dc421e35519d1c', 'b48fadb3eac7c2dfc1da80be4bc9fbf2', '4d4f1d3acb287df727c2839cac676442', '322701a21f259587f7e92f8690f91aed', 'a87c205a8a7d1b5318a53e46439e3880', 'aff5609101b491d28a9f87b891d90291', '329a011b04f8aef22e063cbfda44df80', 'f8d9736da8ce04e71fec0547b6626051', '6e6fd1f8c1909d9a16ffc50c03180b71', 'ac6629b73c2989969310cc665ea95b0d', '429f04f02f3d259344bf453ed52536f8', '33e22b6ad0088ad51682359f14a62e75', '83274dc535a594f4c42b9dae620766e1', 'b8d2994297561cb66e9d7973503dcad7', 'f6439adc4faa188d4ba30d40296f7e5c', '6c101ec46de80b9f535fc9ba3e04c62d', '181b71e92880d3be02b4d9bda7feaa4c', 'cc9f9bf7fa0488d73ccea787e6166219', '9a0323e220914088050d3a82489c3d16', '1f3e5d2a1e740f71ed1654b339e46384', '216dd17a91843c6ae22fb2767495eb92', 'b37be3c8f19410b3

In [ ]:
!yolo task=detect mode=predict source={main_dir+main_exp_dir+iteration_folder+expName+'_Nominees/images/'} model={weights_path} imgsz=256 plots=False  save_txt=True save_conf=True save_crop=False

Ultralytics YOLOv8.0.9 🚀 Python-3.8.10 torch-1.13.1+cu116 CPU
Fusing... 
Model summary: 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
WARNING ⚠️ --img-size [240] must be multiple of max stride 32, updating to [256]
image 1/2830 /content/drive/MyDrive/Smartathon/WRP/Iteration_16/iterationSumAvg16_6000_Nominees/images/000b0fd767c5ab2656b7ed8d43674777.jpg: 160x256 1 garbage, 92.6ms
image 2/2830 /content/drive/MyDrive/Smartathon/WRP/Iteration_16/iterationSumAvg16_6000_Nominees/images/000ed1547634a24f09f22530065d46c9.jpg: 160x256 117.7ms
image 3/2830 /content/drive/MyDrive/Smartathon/WRP/Iteration_16/iterationSumAvg16_6000_Nominees/images/0019a9ab6fd5ef3df915241bc323afb5.jpg: 160x256 1 construction_road, 74.7ms
image 4/2830 /content/drive/MyDrive/Smartathon/WRP/Iteration_16/iterationSumAvg16_6000_Nominees/images/00654503e1204235de86e81d948d0562.jpg: 160x256 3 garbages, 70.6ms
image 5/2830 /content/drive/MyDrive/Smartathon/WRP/Iteration_16/iterationSumAvg16_6000_Nominees/images/0

#Weighing the uncertinities

In [ ]:
expfName = 'iteration'+str(iteration)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)
predFile = '../runs/detect/predict'
wafile = '../runs/detect/val2/results.txt'


In [ ]:
previousFileList = glob.glob(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/*.txt')
trainOnSumAvg = getUncertinitiesForDir(predFile,valAccuracyDirF= wafile, method='sumAvg',budget = int(budgetPerIteration),weighting = 3,)

0.696
graffiti
0
0.208
potholes
2
0.487
garbage
3
0.45
construction_road
4
0.611
bad_billboard
7
0.558
sand_on_road
8
0.834
clutter_sidewalk
9
{'0': 0.16546762589928055, '1': 1.0, '2': 0.750599520383693, '3': 0.4160671462829736, '4': 0.4604316546762589, '5': 1.0, '6': 1.0, '7': 0.2673860911270983, '8': 0.3309352517985611, '9': 0.0, '10': 1.0}
{'bfd7743ad2322a2ef9b948611b23dc15': 0.13197482254196644, '0019a9ab6fd5ef3df915241bc323afb5': 0.08764546762589928, '8bbad1a1b6c91ee4a49354d32656126c': 0.35240003477218224, 'c1b684bd98c8c43ee4edf7e3c4046e25': 0.28295187170263786, '11e6e8a235a5f597a33a731a55fc8025': 0.0, 'c9c24a5ecd2417ef4ef4e0f2f5a5e5d8': 0.5964400000000001, '01db98e38ce4928d94b8b709e4c3745c': 0.2448636258992805, '96974d7fa328ec6e54adf2938acb3dde': 0.0, 'a754f973bdb1298436cca8e907c8c249': 0.1950724158273381, '03ea809773b3e96af83efff9b6182d98': 0.04186595683453237, 'f33b241d2cae631984793e5f8c6ea573': 0.21150627757793763, '9225a326544bbed1984a880f577b6b75': 0.0440640287769784, 'a69d9

#Adding some of the images with undetected lables (As they have inf uncertinity)

In [ ]:
detected = [Path(pth).name[:-4] for pth in glob.glob(predFile+'/labels/*.txt')]
print(len(detected))
allNotTrained = [Path(pth).name[:-4] for pth in (glob.glob(predFile+'/*.jpg'))]
print(len(allNotTrained))

2381
2830


In [ ]:
undetected = list(set(allNotTrained) - set(detected))

#Construct the new train set for the next iteration

In [ ]:
newTraining(expfName,undetected,previousFileList)

449
449


In [ ]:
path_label = complete_iteration_folder+expfName+'/labels/'
path_images = complete_iteration_folder+expfName+'/images/'

wimg = glob.glob(complete_iteration_folder+wexpName+'_Nominees/images/*')
wtxt = glob.glob(complete_iteration_folder+wexpName+'_Nominees/labels/*')
for pth in wtxt:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/labels/'+ namef+'.txt'
  dst_path = path_label + namef+'.txt'
  shutil.copy(src_path, dst_path)
for pth in wimg:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/images/'+ namef+'.jpg'
  dst_path = path_images + namef+'.jpg'
  shutil.copy(src_path, dst_path)

In [ ]:
orp2 = glob.glob(complete_iteration_folder+expfName+'/images/*.jpg')
print(len(orp2))

4609


In [ ]:
from google.colab import runtime
runtime.unassign()

#Getting the images underrepresented classes (done once) to be augmented on RoboFlow

In [ ]:
def containsUnderRepresentedClasses(namedir):
  urc = [10,1,5,2,6,8]
  try:
    with open(namedir, 'r') as file2:  # read annotation.txt
      for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
          if int(row[0]) in urc:
            print(row[0])
            return True
            file2.close()
      return False
  except (IOError, OSError) as e:
      return False


In [ ]:
containsUnderRepresentedClassesList = []
nominatedName = [Path(namedir).name[:-4] for namedir in glob.glob(main_dir+main_exp_dir+iteration_folder+expName+'_Nominees/images/*.jpg')]
nominatedDir = [complete_data_folder+'labels/'+ namef+'.txt' for namef in nominatedName]
nnomD = glob.glob(complete_data_folder+'/labels/*.txt')
c = 0
f = 0
for namedir in nnomD:

  f = f + 1
  if containsUnderRepresentedClasses(namedir):
    c = c + 1
    containsUnderRepresentedClassesList.append(Path(namedir).name[:-4])
  

2
9 1
2
19 2
2
23 3
1
25 4
2
29 5
2
34 6
2
35 7
8
38 8
2
39 9
8
46 10
2
54 11
2
60 12
2
67 13
2
78 14
2
81 15
2
88 16
2
100 17
2
105 18
2
106 19
5
107 20
8
110 21
2
112 22
2
114 23
2
124 24
2
126 25
2
128 26
8
130 27
2
133 28
2
137 29
2
143 30
2
144 31
2
153 32
2
159 33
8
165 34
2
168 35
2
169 36
2
171 37
8
177 38
2
178 39
8
180 40
8
183 41
2
192 42
2
195 43
10
196 44
8
200 45
2
201 46
8
202 47
2
213 48
2
220 49
8
225 50
8
227 51
2
228 52
2
238 53
8
244 54
8
246 55
2
257 56
2
258 57
2
260 58
2
264 59
8
270 60
2
271 61
2
273 62
8
274 63
2
275 64
10
276 65
2
279 66
2
284 67
2
288 68
8
290 69
2
293 70
2
294 71
2
299 72
8
304 73
8
308 74
2
309 75
2
314 76
2
315 77
2
316 78
2
317 79
8
336 80
2
348 81
2
350 82
8
356 83
2
357 84
8
362 85
8
364 86
2
365 87
2
369 88
2
371 89
2
382 90
2
383 91
2
387 92
2
390 93
2
391 94
2
404 95
8
412 96
2
413 97
8
416 98
2
419 99
8
431 100
2
432 101
8
437 102
2
445 103
8
451 104
2
462 105
2
463 106
2
466 107
8
468 108
2
472 109
8
477 110
8
480 111
2
482 112
8
4

In [ ]:
len(containsUnderRepresentedClassesList)

1635

In [ ]:

os.makedirs(complete_iteration_folder+'rare/labels/') if not os.path.exists(complete_iteration_folder+'rare/labels/') else None
os.makedirs(complete_iteration_folder+'rare/images/') if not os.path.exists(complete_iteration_folder+'rare/images/') else None

for rareName in containsUnderRepresentedClassesList:

  src_path = complete_data_folder+'labels/'+ rareName+'.txt'
  print(src_path)
  dst_path = complete_iteration_folder+'rare/labels/' + rareName+'.txt'
  shutil.copy(src_path, dst_path)

  src_path = complete_data_folder+'images/'+ rareName+'.jpg'
  dst_path = complete_iteration_folder+'rare/images/' + rareName+'.jpg'
  shutil.copy(src_path, dst_path)


/content/drive/MyDrive/Smartathon/TrainF/labels/6268bd55a8c86072b7c746c29bfe01c1.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/c760b28d8639b875a71c1603930827a4.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/657389175c17dba6920cbb95c43f23b6.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/ba7c3e7cfd9ee44c6d0e64d328091c7b.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/1228ee52547cc41610ebde6c552c204a.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/52e006572945eb3f180c39c8b47edd60.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/2a0309fb866c4344c5105383bb662ba9.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/f07921ad2ce280ceb6011b5bbb032918.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/52e830f16eb702127491074fb7e84ed4.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/6d0748bc8874bdcba93a419bc4821c49.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/6bd0e7c1c1f30902611eddc0e41338a2.txt
/content/drive/MyDrive/Smartathon/TrainF/labels/9876c6d2f99cc13d4